In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.set_option('float_format', '{:.2f}'.format)
plt.style.use('seaborn-colorblind')
cm = sns.light_palette("green", as_cmap=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Run bean-query to generate table
!bean-query -q ~/finance/money.beancount "SELECT year, month, root(account, 2) as Account, sum(number) as Total, currency WHERE account ~ 'Expenses' OR account ~ 'Liabilities:UK:HSBC:Mortgage' OR account ~ 'Income' GROUP BY year, month, Account, currency ORDER BY year, month, currency, Account FLATTEN" > monthly.txt

In [29]:
# Process Data
df = pd.read_csv("monthly.txt", delim_whitespace=True)
df['Type'], df['Category'] = df['account'].str.split(':', 1).str
df = df.drop(df.index[0])
df = df.drop(['account'], axis=1)
df = df.rename(columns={'year':'Year','mo':'Month','total':'Net','cur':'Currency',})
df = df[['Year', 'Month', 'Type', 'Category', 'Net', 'Currency']]
df['Net'] = pd.to_numeric(df['Net'])
df['Net'] = df['Net'] * -1  # Make it normal

#Fix mortgage category
df.loc[df['Type'] == "Liabilities", ['Category']] = "Home"
df['Type'] = df['Type'].replace(['Liabilities'], 'Expenses')
df.head()

,Year,Month,Type,Category,Net,Currency
1,2014,10,Expenses,Fees,-29.94,GBP
2,2015,1,Expenses,Gift,-399.71,GBP
3,2015,1,Expenses,Home,-19.90,GBP
4,2015,1,Expenses,NBC,-150.55,GBP
5,2015,1,Expenses,Shopping,136.92,GBP


In [32]:
# Summary Table
#exp_summary = pd.pivot_table(df.query('Type == ["Expense"]'), index=['Master Category'], columns=['Month'], values=['Net'], aggfunc=np.sum, fill_value=0)
summary = pd.pivot_table(df.query('Year == ["2017"]'), index=['Type','Category','Currency'], columns=['Month'], values=['Net'], aggfunc=np.sum, fill_value=0)
#summary['Total'] = summary.sum(axis=1) # adds total to categories
#summary.loc['Total']= summary.sum() # adds total row at bottom
summary

Net                                    
Month                                   1        2        3        4        5
Type     Category      Currency                                              
Expenses Car           GBP           0.00     0.00     0.00  -137.30     0.00
                       USD      -18766.10  -259.99  -363.12  -161.30     0.00
         Entertainment USD         -51.50   -14.99   -59.97   -39.00     0.00
         Fees          GBP        -213.90     0.00   -30.38     0.00     0.00
                       USD          -7.77   -14.35     1.05    -1.05     4.58
         Food          GBP         -77.33     0.00   -30.00     0.00     0.00
                       USD       -1019.69  -569.30  -673.71 -1051.58     0.00
         Gift          GBP         -27.60   -84.40   -72.00   -41.53     0.00
                       USD         -54.79  -157.74  -292.02   -10.53     0.00
         Health        USD         -96.47     0.00   -31.11   -18.00     0.00
         Home          GBP        -949.07  -947.78  -928.93  -945.05  -756.30
                       USD       -2946.96 -2928.69 -2825.87 -2918.04 -2400.00
         Kids          GBP        -100.00  -135.33  -100.00  -685.69  -303.00
                       USD        -530.86  -534.00  -574.09  -150.00   587.00
         NBC           GBP        -125.69  -412.34   -37.01  -478.01   -96.00
         Pets          USD           0.00     0.00   -75.50     0.00     0.00
         Shopping      GBP        -340.94    11.42  -127.56     0.00     0.00
                       USD        -859.75   -28.60 -1412.44  -189.62     0.00
         Travel        USD           3.88     0.00   -94.00     0.00     0.00
Income   Gift          GBP        2000.00     0.00     0.00   200.00     0.00
         MoneyOwed     GBP         300.00     0.00     0.00     0.00     0.00
                       USD           6.97     0.00     0.00     0.00     0.00
         UK            GBP       15168.61  7294.25 10396.43  7194.36     0.00
         US            USD        3005.34  2991.50  2992.13  3125.13     0.00